In [11]:
training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

header = ["color", "diameter", "label"]

# Find the unique values for a column in a dataset
def unique_vals(rows, col):
    return set([row[col] for row in rows])

# Counts the number of each type of example in a dataset
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0 
        counts[label] += 1
        return counts

# Test whether a value is numeric or not
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

class Question:
    """
    This class records a column number e.g 0 for color and column value e.g Green.
    The match method is used to compare the feature value in an example 
    to the feature value stored in the questions
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        # This is just a helper method to print the question in a readable format
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
    
def partition(rows, question):
    """
    Partions a dataset.
    For each row in the dataset, check if it matches the question. If so, add it
    to 'true rows', otherwise, add it to 'false rows'
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
        return true_rows, false_rows

def gini(rows):
    # Calculate the Gini Impurity for a list of rows
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    # The Uncertainty of the starting node, minus weighted impurity of 2 child nodes
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

def find_best_split(rows):
    """Find the best split to ask by iterating over every feature/value 
    and calculating the info gain"""
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    
    for col in range(n_features): # for each feature
        values = set([row[col] for row in rows]) # unique values in the column
        for val in values: # for each value
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            # skip the split if it doesnt divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            # calculate info gain from the split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            # It's more prudent to use > instead of >= here but this is a toy dataset
            # If it works then it's fine I guess
            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question


class Leaf:
    """This node classifies data
    It holds a dictionary of class e.g Mango -> number of times 
    it appears in the rows from the training data that reach this leaf"""
    def __init__(self, rows):
        self.predictions = class_counts(rows)


class Decision_Node:
    """A decision node asks a question.
    It holds a reference to the question and to the two child nodes"""
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


def build_tree(rows):
    """Builds the tree. Let's see if it works..."""
    # Try partitioning the dataset on each of the unique attributes
    # Calculate the info gain
    # Return the question that produces the highest gain
    gain, question = find_best_split(rows)
    
    # Base case(no further info gain)
    # Since we can ask no further questions, we return a leaf
    if gain == 0:
        return Leaf(rows)
    
    # Reaching here means we've found a useful feature/value to partition on
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch
    false_branch = build_tree(false_rows)
    
    # Return a Question node
    # This records the best feature/value to ask at this point,
    # as well as the branches to follow depending on the answer
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):
    """Lets see if we can visualize it manually"""
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    # Print the question at this node
    print (spacing + str(node.question))
    
    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    
    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + " ")
    

def classify(row, node):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true branch/false branch
    # Compare the feature/value stored in the node to the example we're considering
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


def print_leaf(counts):
    """Print the predictions at a leaf"""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs


In [12]:
if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    
    # Evaluate
    testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 4, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]
    for row in testing_data:
        print("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))
        

Predict {'Mango': 1}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Grape. Predicted: {'Mango': '100%'}
Actual: Grape. Predicted: {'Mango': '100%'}
Actual: Lemon. Predicted: {'Mango': '100%'}


In [ ]:
# This hasnt perfomed that well so maybe we could add more rows to the dataset
# There is also a bug in the __repr__ method or the print_tree method